# 1) Introduction

![](https://katbailey.github.io/images/matrix_factorization.png)

- Rating matrix(NxM)
- N = num users
- M = num items

## Matrix Factorization 
- In supervised ML, we want accuracy(predictions close to target)
- In recommenders, what we want is a score to sort recommendations by


## Section Outline
- Basic form of Matrix Factorization model
- Define a loss, minimize it
- 2 impl
    - numpy - direct from theory
    - Keras
- Extend keras model


# 2) Matrix Factorization 

## Factors
- 10 = 5 x2 
- 15 = 3 x 5
- 30 = 3 x 10 = 15 x 2

## Matirx Factorization 

- Split the matrix into the product of 2 other matrices
- R hat is approximates R - it is our model of R
![](https://www.kukuxiaai.com/images/blog/recommended_system/udemy/mf_1.png)

- W( N x K) - user matrix, U (M x K) - movie matrix
- K somewhere from 10 -50

## think about R
- W and U should be much smaller than R
- R is N x M
- represent it using a special data structure
    - Dict{(u,m) -> r}
- If N = 130k, M = 26k
    - N x M = 3.38 billion
    - ratings = 20 million
    - space used: 20 million/3.38billion = 0.006
- This is called a sparse representation

![](https://4.bp.blogspot.com/-95QD5t9Lha4/Wd7uWnBZBeI/AAAAAAAADg4/xB4VnnxM0UgUp15lNmB3aHCXYGejpm4OACLcBGAs/s1600/matrix_factorization.png)

## Some calculations
- If k = 10, N = 130k, M = 26k, then size of W and U 
- NK + MK = 1.56 million
- how much savings?
- 1.56 million / 3.38 billion = 0.0005
- this is good, we like # parameters < # of data pts)


- What happens if you try to calculate W$U^T$ in code?
- Don't do it, the result is NxM, which I just told you is exactly what we don't want
    - Unless you've selected a small subset of your data

## One rating
- this is easy, just a dot product between 2 vectors of size K

\begin{equation*}
\hat{r}_{ij} = w_{i}^{T}u_{j}, \hat{r}_{ij} = \hat{R}[i,j] , w_{i} = W[i], u_{j} = U[j]
\end{equation*}

## Why dose it make sense?
- From a mathmetical standpoint, we know from SVD(singular value decomposition) that a matrix X can be decomposed into 3 seperate matrices multiplied together

- X(N x M), U(N x K), S(K x K) , V(M x K)
- If I multiply U by S, I just get another N x K matrix
    - Then X is a product of 2 matrices, just like matrix factorization
    - or equivalently, I could combine S with $V^{T}$
- R(rating matrix) is sparse
    - If U,S and V can properly approximate a full X matrix, then surely it can approximate a mostly empty R matrix 
    
\begin{equation*}
X = USV^{T}
\end{equation*}

![](http://cdn.app.compendium.com/uploads/user/e7c690e8-6ff9-102a-ac6d-e4aebca50425/f4a5b21d-66fa-4885-92bf-c4e81c06d916/Image/229f77d2cb173c1cef4d6cfbab2e905e/svd_matrices.jpg)

## Interpretation
- Each of the K elements in $W_{i}$ and $u_{j}$ is a feature
- Let's suppose K=5, and they are
    - Action/adventure
    - Comedy
    - Romance
    - Horror
    - Animation
- $w_{i}(1)$ is how much user i likes action
- $w_{i}(2)$ is how much user i likes comedy 
- $u_{j}(1)$ is how much movie j contains action
- $u_{j}(2)$ is how much movie j contains comedy



- What happens when we dot $w_{i}^{T}u_{j}$ ?
- How well do user i's preferences correlate with movie j's attributes?

\begin{equation*}
w_{i}^{T}u_{j} = ||w_{i}||||u_{j}|| cos\theta \propto sim(i,j)
\end{equation*}

## Example
- Action/adventure
- Comedy
- Romance
- Horror
- Animation

- $w_{i}$ = (1, 0.8, -1, 0.1, 1)
- $u_{j}$ = (1,1.5,-1.3,0, 1.2)
- result = 1 * 1 + 0.8 * 1.5 + 1 * 1.3 + 0.1 * 0 + 1 * 1.2 = 4.7 (too high)
- Why?
    - +ve x +ve -> +ve
    - -ve x -ve -> +ve


- $w_{i}$ = (1, 0.8, -1, 0.1, 1)
- $u_{j}$ = (-1,-1,1,0, -1)
- result = 1 * -1 + 0.8 * -1 + -1 * 1 + 0.1 * 0 + 1 * -1 = -3.8 (too low)
- Why?
    - +ve x -ve -> -ve



## Features

- You can't choose feature 1 to be action, feature 2 to be comedy
- Each feature is latent, and K is the latent dimensionality
- Hidden causes
- Why user i like Power rangers?
    - hidden cause is that user i likes action, and Power rangers has action
- We don't know the meaning of any feature without inspecting it
- Ex. check top 10 movies that have the largest value for feature 1 


## Supervised machine learning

- recall our previous discussion, we could predict how much a user likes an item, by extracting features from both, and feeding it into a model like random forest or neural network
- the difference is that Matrix Fatorization extracts the features automatically using only ratings


## Dimensionality Reduction

![](https://www.kukuxiaai.com/images/blog/recommended_system/udemy/mf_2.png)

# 3) Training

- How can we ensure our approximation is good?

$R \approx \hat{R} = WU^{T}$

## Squared error loss

\begin{equation*}
J = \sum_{i,j\in\Omega}^{}(r_{ij}-\hat{r}_{ij})^2 = \sum_{i,j\in\Omega}^{}(r_{ij}-w_{i}^{T}u_{j})^2
\end{equation*}

$\Omega$ = set of pairs(i,j) where user i rated movie j

## Minimize the loss
- How? Find the gradient, set it to 0, solve for the parameters

## Solving for W

- careful about which sets are being summed over
- For J, we want to sum over all ratings
- For a particular user vector $w_{i}$, we only care about movies that user rated 
- Try to isoloate $w_{i}$
- it's stuck inside a dot product 

\begin{equation*}
\frac{\partial J}{\partial w_{i}} = 2 \sum_{j\in \Psi_{i}}^{}(r_{ij} - w_{i}^{T}u_{j})(-u_{j}) = 0  \\
\sum_{j\in\Psi_{i}}^{}(w_{i}^{T}u_{j})u_{j} = \sum_{j\in\Psi_{i}}^{}r_{ij}u_{j} \\
\sum_{j\in\Psi_{i}}^{}(u_{j}^{T}w_{i})u_{j} = \sum_{j\in\Psi_{i}}^{}r_{ij}u_{j}
\end{equation*}

- scalar x vector = vector x scalar

\begin{equation*}
\sum_{j\in\Psi_{i}}^{}u_{j}(u_{j}^{T}w_{i}) = \sum_{j\in\Psi_{i}}^{}r_{ij}u_{j}
\end{equation*}

- drop the brackets

\begin{equation*}
\sum_{j\in\Psi_{i}}^{}u_{j}u_{j}^{T}w_{i} = \sum_{j\in\Psi_{i}}^{}r_{ij}u_{j}
\end{equation*}

\begin{equation*}
(\sum_{j\in\Psi_{i}}^{}u_{j}u_{j}^{T})w_{i} = \sum_{j\in\Psi_{i}}^{}r_{ij}u_{j}
\end{equation*}

- Now it's just Ax = b, which we know how to solve
- x = np.linalg.solve(A,b)

\begin{equation*}
w_{i} = (\sum_{j\in\Psi_{i}}^{}u_{j}u_{j}^{T})^{-1} \sum_{j\in\Psi_{i}}^{}r_{ij}u_{j}
\end{equation*}


## Solving for U

- symmetric in W and U, so the steps should be the same
\begin{equation*}
\frac{\partial J}{\partial u_{j}} = 2 \sum_{i\in \Omega_{j}}^{}(r_{ij} - w_{i}^{T}u_{j})(-w_{i}) = 0  \\
\sum_{i\in\Omega_{j}}^{}(w_{i}^{T}u_{j})w_{i} = \sum_{i\in\Omega_{j}}^{}r_{ij}w_{i}\\
\sum_{i\in\Omega_{j}}^{}w_{i}w_{i}^{T}u_{j} = \sum_{i\in\Omega_{j}}^{}r_{ij}w_{i}\\
(\sum_{i\in\Omega_{j}}^{}w_{i}w_{i}^{T})u_{j} = \sum_{i\in\Omega_{j}}^{}r_{ij}w_{i}\\
u_{j} = (\sum_{i\in\Omega_{j}}^{}w_{i}w_{i}^{T})^{-1} \sum_{i\in\Omega_{j}}^{}r_{ij}w_{i}
\end{equation*}


## 2-way dependency
- solution for W depends on U
- solution for U depends on W

 ## Training Algorithm
 
 - W = randn(N,K) U = randn(M,K)
 - for t in range(T):

\begin{equation*}
w_{i} = (\sum_{j\in\Psi_{i}}^{}u_{j}u_{j}^{T})^{-1} \sum_{j\in\Psi_{i}}^{}r_{ij}u_{j} \\
u_{j} = (\sum_{i\in\Omega_{j}}^{}w_{i}w_{i}^{T})^{-1} \sum_{i\in\Omega_{j}}^{}r_{ij}w_{i}
\end{equation*}


## FAQ
- Does it matter which order you update in? it doesn't matter
- Should you use the old values of W when updating U?
    - Tends to go faster if you use the new values
    - computationally, if you wanted to use the old values, you'd have to make a copy(very slow)

# 4) Matrix Factorization , Expanding our model

## Bias Terms
- It thus makes sense to add bias terms to the MF model

\begin{equation*}
\hat{r}_{ij} = w_{i}^{T}u_{j} + b_{i}+ c_{j}+ \mu \\
\end{equation*}
$b_{i}$ = user bias  
$c_{j}$ = movie bias  
$\mu$ = global average  

## Training

\begin{equation*}
J = \sum_{i,j\in\Omega}^{}(r_{ij}-\hat{r}_{ij})^2 \\
\hat{r}_{ij} = w_{i}^{T}u_{j}+ b_{i}+c_{j}+ \mu
\end{equation*}




## Solving for W

\begin{equation*}
\frac{\partial J}{\partial w_{i}} = 2 \sum_{j\in\Psi_{i}}^{}(r_{ij}-w_{i}^{T}u_{j}-b_{i}-c_{j}-\mu)(-u_{j}) = 0 \\
\sum_{j\in\Psi_{i}}^{}(w_{i}^{T}u_{j})u_{j} = \sum_{j\in\Psi_{i}}^{}(r_{ij}-b_{i}-c_{j}-\mu)u_{j} \\
w_{i} = (\sum_{j\in\Psi_{i}}^{}u_{j}u_{j}^{T})^{-1}  \sum_{j\in\Psi_{i}}^{}(r_{ij}-b_{i}-c_{j}-\mu)u_{j}
\end{equation*}

## Solving for U

\begin{equation*}
u_{j} = (\sum_{i\in\Omega_{j}}^{}w_{i}w_{i}^{T})^{-1}  \sum_{i\in\Omega_{j}}^{}(r_{ij}-b_{i}-c_{j}-\mu)w_{i}
\end{equation*}

## Solving for b

\begin{equation*}
\frac{\partial J}{\partial b_{i}} = 2 \sum_{j\in\Psi_{i}}^{}(r_{ij}-w_{i}^{T}u_{j}-b_{i}-c_{j}-\mu)(-1) = 0 \\
b_{i} = \frac{1}{|\Psi_{i}|}\sum_{j\in\Psi_{i}}^{}(r_{ij}-w_{i}^{T}u_{j}-c_{j}-\mu)
\end{equation*}

## Solving for c

\begin{equation*}
\frac{\partial J}{\partial c_{j}} = 2 \sum_{i\in\Omega_{j}}^{}(r_{ij}-w_{i}^{T}u_{j}-b_{i}-c_{j}-\mu)(-1) = 0 \\
c_{j} = \frac{1}{|\Omega_{j}|}\sum_{i\in\Omega_{j}}^{}(r_{ij}-w_{i}^{T}u_{j}-c_{j}-\mu)
\end{equation*}

- Don't need to update global average(just calculate it directlry from train data)

# 5) Matrix Factorization ,Regularization

## Regularization
- A technique to prevent overfitting and help generalization
- In linear regression
- Model $\hat{y} = w^{T}x$
- Objective $J = \sum_{i=1}^{N}(y_{i}-\hat{y}_{i})^2 +\lambda||w||_{2}^{2}$
- Solution $ w = (\lambda I + X^{T}X)^{-1}X^{T}y$

## Regularization in Matrix Factorization
- Same approach, add squared magnitude of each parameter multiplied by regularization constant
- $||*||_{F}$ is called the Frobenius norm

$J = \sum_{i,j\in \Omega}^{}(r_{ij}-\hat{r}_{ij})^2 +\lambda(||W||_{F}^{2}+||U||_{F}^{2}+||b||_{2}^{2}+||c||_{2}^{2})$

## Solve for W
- Derivatives are additive, we just need to differentiate the 2nd term and add it to the existing derivative

\begin{equation*}
\frac{\partial J}{\partial w_{i}} = 2\sum_{j\in \Psi_{i}}^{}(r_{ij}-W_{i}^{T}u_{j}- b_{i}-c_{j}-\mu)(-u_{j})+2\lambda w_{i} = 0
\end{equation*}

- If you can't see how I differentiated $w_{i}$ wrt Frobenius Norm, expand it
- Now it's just a dot product which we know how to differentiate

\begin{equation*}
||W||_{F}^{2} = \sum_{i=1}^{N}\sum_{k=1}^{K}|w_{ik}|^{2} = \sum_{i=1}^{N}||w_{i}||_{2}^{2}= \sum_{i=1}^{N}w_{i}^{T}w_{i}
\end{equation*}

\begin{equation*}
\sum_{j\in \Psi_{i}}^{}u_{j}u_{j}^{T}w_{i}+ \lambda w_{i} =\sum_{j\in \Psi_{i}}^{} (r_{ij}- b_{i}-c_{j}-\mu)(u_{j}) \\
(\sum_{j\in \Psi_{i}}^{}u_{j}u_{j}^{T}+ \lambda I) w_{i} =\sum_{j\in \Psi_{i}}^{} (r_{ij}- b_{i}-c_{j}-\mu)u_{j} \\
w_{i} = (\sum_{j\in \Psi_{i}}^{}u_{j}u_{j}^{T}+ \lambda I)^{-1}\sum_{j\in \Psi_{i}}^{} (r_{ij}- b_{i}-c_{j}-\mu)u_{j}
\end{equation*}

## Solve for U 
\begin{equation*}
u_{j} = (\sum_{i\in \Omega_{j}}^{}w_{i}w_{i}^{T}+ \lambda I)^{-1}\sum_{i\in \Omega_{j}}^{} (r_{ij}- b_{i}-c_{j}-\mu)w_{i}
\end{equation*}

## Solev for b

\begin{equation*}
\frac{\partial J}{\partial b_{i}} = 2\sum_{j\in \Psi_{i}}^{}(r_{ij}-w_{i}^{T}u_{j}-b_{i}- c_{j}-\mu)(-1)+2\lambda b_{i} = 0 \\
\sum_{j\in \Psi_{i}}^{}b_{i}+\lambda b_{i} = \sum_{j\in \Psi_{i}}^{}(r_{ij}-w_{i}^{T}u_{j} -c_{j}-\mu) \\
b_{i}((\sum_{j\in \Psi_{i}}^{}1) + \lambda) = \sum_{j\in \Psi_{i}}^{}(r_{ij}-w_{i}^{T}u_{j} -c_{j}-\mu) \\
b_{i} = \frac{1}{|\Psi_{i}|+\lambda}\sum_{j\in \Psi_{i}}^{}(r_{ij}-w_{i}^{T}u_{j} -c_{j}-\mu)
\end{equation*}

## Solev for c

\begin{equation*}
c_{j} = \frac{1}{|\Omega_{j}|+\lambda}\sum_{i\in \Omega_{j}}^{}(r_{ij}-w_{i}^{T}u_{j} -b_{i}-\mu)
\end{equation*}

# 6) Matrix Factorization in Code

```python
# N is user number, M is number of movies
N = np.max(list(user2movie.keys())) + 1
# the test set may contain movies the train set doesn't have data on
m1 = np.max(list(movie2user.keys()))
m2 = np.max([m for (u, m), r in usermovie2rating_test.items()])
M = max(m1, m2) + 1
print("N:", N, "M:", M)


```

$w_{i}$ = user i  
$u_{j}$ = movie j  
$b_{i}$ = user bias  
$c_{j}$ = movie bias  
$\mu$ = global average  

\begin{equation*}
\hat{r}_{ij} = w_{i}^{T}u_{j} + b_{i}+ c_{j}+ \mu \\
\end{equation*}

```python
# initialize variables
K = 10 # latent dimensionality
W = np.random.randn(N, K)
b = np.zeros(N)
U = np.random.randn(M, K)
c = np.zeros(M)
mu = np.mean(list(usermovie2rating.values()))
# prediction[i,j] = W[i].dot(U[j]) + b[i] + c.T[j] + mu
```

\begin{equation*}
J = \sum_{i,j\in\Omega}^{}(r_{ij}-\hat{r}_{ij})^2 \\
\hat{r}_{ij} = w_{i}^{T}u_{j}+ b_{i}+c_{j}+ \mu
\end{equation*}

```python
def get_loss(d):
  # d: (user_id, movie_id) -> rating
  N = float(len(d))
  sse = 0
  for k, r in d.items():
    i, j = k
    p = W[i].dot(U[j]) + b[i] + c[j] + mu
    sse += (p - r)*(p - r)
  return sse / N
```


\begin{equation*}
w_{i} = (\sum_{j\in\Psi_{i}}^{}u_{j}u_{j}^{T})^{-1}  \sum_{j\in\Psi_{i}}^{}(r_{ij}-b_{i}-c_{j}-\mu)u_{j}\\
w_{i} = (\sum_{j\in \Psi_{i}}^{}u_{j}u_{j}^{T}+ \lambda I)^{-1}\sum_{j\in \Psi_{i}}^{} (r_{ij}- b_{i}-c_{j}-\mu)u_{j}\\
b_{i} = \frac{1}{|\Psi_{i}|}\sum_{j\in\Psi_{i}}^{}(r_{ij}-w_{i}^{T}u_{j}-c_{j}-\mu)
\end{equation*}

$(\sum_{j\in\Psi_{i}}^{}u_{j}u_{j}^{T})^{-1}$ is matrix  
$\sum_{j\in\Psi_{i}}^{}(r_{ij}-b_{i}-c_{j}-\mu)u_{j}$ is vector  
Ax = b  

```python
for i in range(N):
    # for W
    matrix = np.eye(K) * reg
    vector = np.zeros(K)

    # for b
    bi = 0
    for j in user2movie[i]:
      r = usermovie2rating[(i,j)]
      matrix += np.outer(U[j], U[j])
      vector += (r - b[i] - c[j] - mu)*U[j]
      bi += (r - W[i].dot(U[j]) - c[j] - mu)

    # set the updates
    W[i] = np.linalg.solve(matrix, vector)
    b[i] = bi / (len(user2movie[i]) + reg)
```

```python
# another implementation
for i in range(N):
    m_ids, r = user2movierating[i]
    matrix = U[m_ids].T.dot(U[m_ids]) + np.eye(K) * reg
    vector = (r - b[i] - c[m_ids] - mu).dot(U[m_ids])
    bi = (r - U[m_ids].dot(W[i]) - c[m_ids] - mu).sum()

    # set the updates
    W[i] = np.linalg.solve(matrix, vector)
    b[i] = bi / (len(user2movie[i]) + reg)

```

\begin{equation*}
u_{j} = (\sum_{i\in\Omega_{j}}^{}w_{i}w_{i}^{T})^{-1}  \sum_{i\in\Omega_{j}}^{}(r_{ij}-b_{i}-c_{j}-\mu)w_{i}\\
c_{j} = \frac{1}{|\Omega_{j}|}\sum_{i\in\Omega_{j}}^{}(r_{ij}-w_{i}^{T}u_{j}-c_{j}-\mu)
\end{equation*}


```python
# update U and c
  t0 = datetime.now()
  for j in range(M):
    # for U
    matrix = np.eye(K) * reg
    vector = np.zeros(K)

    # for c
    cj = 0
    try:
      for i in movie2user[j]:
        r = usermovie2rating[(i,j)]
        matrix += np.outer(W[i], W[i])
        vector += (r - b[i] - c[j] - mu)*W[i]
        cj += (r - W[i].dot(U[j]) - b[i] - mu)

      # set the updates
      U[j] = np.linalg.solve(matrix, vector)
      c[j] = cj / (len(movie2user[j]) + reg)

      if j % (M//10) == 0:
        print("j:", j, "M:", M)
    except KeyError:
      # possible not to have any ratings for a movie
      pass
```

```python
# another implementation
for j in range(M):
    try:
      u_ids, r = movie2userrating[j]
      matrix = W[u_ids].T.dot(W[u_ids]) + np.eye(K) * reg
      vector = (r - b[u_ids] - c[j] - mu).dot(W[u_ids])
      cj = (r - W[u_ids].dot(U[j]) - b[u_ids] - mu).sum()

      # set the updates
      U[j] = np.linalg.solve(matrix, vector)
      c[j] = cj / (len(movie2user[j]) + reg)

```

![](https://www.w3resource.com/w3r_images/linear-algebra-image-exercise-2.png)

# 7) SVD(Singular Value Decomposition)

## SVD
- X can be decomposed into a product of U,S,V
- X(N x M),U(N x M), S(M x M) , V(M x M)
- No immediate savings.. size(X) == size(U)

$X=USV^{T}$

![](http://slideplayer.com/slide/3900405/13/images/25/Singular+Value+Decomposition.jpg)
![](https://image.slidesharecdn.com/4772391/95/building-a-recommendation-engine-an-example-of-a-product-recommendation-engine-21-728.jpg?cb=1279279430)

## Truncated SVD
- U,S and V are ordered by importance
- Thus, it's possible to cut off parts of U,S, and V to get the best possible low rank approximation of X
- U(N x K), S(K x K), V(M x K) -> result is still N x M

## SVD 
- What's the justification for calling matrix factorization "SVD"?
- S can be absorbed into U or V, we get an (N x K)x(K x M) -> (N x M)
- The model is thus exactly like MF  
\begin{equation*}
\hat{x}_{ij} = \sum_{k}^{}u_{ik}s_{kk}v_{kj} = \sum_{k}^{}(u_{ik}s_{kk})v_{kj} = \sum_{k}^{}u'_{ik}v_{kj} = u_{i}'^{T}v_{j}
\end{equation*}

![](http://shakthydoss.com/wp-content/uploads/2014/06/Singular-value-decomposition.jpg)
![](http://image.slideserve.com/244082/singular-value-decomposition-l.jpg)

- But how does SVD actually work? How do we find U,S and V?
- Hint: not by setting the gradient to 0
- Rough outline
    - X is N x M
    - Assume N > M , rank of X is M 
    - Find 2 matrices: $X^{T}X$(M x M) and $XX^{T}$(N x N)
    - $X^{T}X$ is proportional to the covariance of X, if X has already been centered
    - another motivation for bias term
    
- Find the eigenvalues and eigenvectors of $X^{T}X$ and $XX^{T}$    
- There will be M of them, and they are the same for both matrices
- Suppose we found all the eigenvalues( np.linalg.eigh)
    - $\lambda_{1},\lambda_{2},...,\lambda_{M}$
- S is just a diagonal matrix of the square root of them
$$
    \begin{pmatrix}
    \sqrt{\lambda_{1}} & 0 & ... & 0 \\
    0 & \sqrt{\lambda_{2}} & ... & 0 \\
    ... & ... & ... & ... \\
    0 & 0 & ... & \sqrt{\lambda_{M}} \\
    \end{pmatrix}  
$$

- for each eigenvalue, there's a corresponding eigenvector
$$
(X^{T}X)v_{i} = \lambda_{i}v_{i}\\
(XX^{T})u_{i} = \lambda_{i}u_{i}\\
$$

- Important result, v's are orthonormal, u's are orthonormal

$$
u_{i}^{T}u_{j} = 0, i \neq j \\
u_{i}^{T}u_{j} = 1 \\
v_{i}^{T}v_{j} = 0, i \neq j \\
v_{i}^{T}v_{j} = 1
$$


- X(N x M),U(N x M), S(M x M) , V(M x M)
- No immediate savings.. size(X) == size(U)

$X=USV^{T}$


## Matrix Factorization
- SVD doesn't even work when X has missing values


# 8) Probabilistic Matrix Factorization

- Start with a probablistic model
- The mean of R is $WU^{T}$

$$
R \sim N(WU^{T},\sigma^{2})\\
r_{ij} \sim N(w_{i}^{T}u_{j},\sigma^{2})
$$

## Maximizing the likelihood

- When we have a probability distribution, and we want to find the parameters of it, how can we do that?
- Maximum likelihood estimation
- collect data from the distribution, maximize the likelihood function wrt parameters

$$
L = \prod_{i,j\in \Omega}\frac{1}{\sqrt{2\pi\sigma^{2}}}exp(\frac{-1}{2\sigma^{2}} (r_{ij}-w_{i}^{T}u_{j})^{2})\\
W,U = \underset{W,U}{\operatorname{argmax}}L
$$

- Usual next step: take the log of the likelihood
- Maximizing this is usually easier
- This is just the negative of our loss

$$
log(L) =C - \prod_{i,j\in \Omega}\frac{1}{2\sigma^{2}}(r_{ij}-w_{i}^{T}u_{j})^{2}\\
$$


## MAP Estimation
- Maximum a posterior estimation

$$
MLE: p(R|W,U) \\
MAP :p(W,U|R)
$$

- Using Bayes rule

$$
p(W,U|R) = \frac{p(R|W,U)p(W)p(U)}{p(R)}
$$

- Thus we must define the priors p(W), and p(U)
- p(R) is a constant wrt W,U, can be dropped

$$
p(W) = N(0,\lambda^{-1})\\
p(U) = N(0,\lambda^{-1})
$$

## MAP Objective
- we want to take the log of this

$$
L = \prod_{i,j\in \Omega}\frac{1}{\sqrt{2\pi\sigma^{2}}}exp(\frac{-1}{2\sigma^{2}} (r_{ij}-w_{i}^{T}u_{j})^{2}) \times  \frac{\lambda}{\sqrt{2\pi}}e^{\frac{-\lambda}{2}||W||_{F}^{2}}\frac{\lambda}{\sqrt{2\pi}}e^{\frac{-\lambda}{2}||U||_{F}^{2}}
$$

- This is just regularized matrix factorization(constant are irrelevant)
$$
L =C_{0}-C_{1} \sum_{i,j\in\Omega}^{}(r_{ij}-w_{i}^{T}u_{j})^{2}- \frac{-\lambda}{2}||W||_{F}^{2} - \frac{-\lambda}{2}||U||_{F}^{2}
$$



# 9) Bayesian Matrix Factorization

- Summarize the Bayesian approach as everything is a random variable

$$
R \sim N(WU^{T},\sigma^{2})\\
r_{ij} \sim N(w_{i}^{T}u_{j},\sigma^{2})
$$

$$
p(W) = N(0,\lambda^{-1})\\
p(U) = N(0,\lambda^{-1})
$$

- Posterior
- This time, we can't ignore p(R), because we don't want the argmax
- Because this is Bayesian, we want the actual distribution p(W,U|R)

$$
p(W,U|R) = \frac{p(R|W,U)p(W)p(U)}{p(R)}
$$


## What is our goal?
- we still want to predict ratings e.g. $r_{ij}$
- since this is Bayesian it'll be $p(r_{ij}|R)$
- R is collected data

## Expected value
- How about $E(r_{ij}|R)$
- In general
$$
E(x) = \int_{-\infty}^{\infty}xp(x)dx
$$

- For us
$$
E(r_{ij}|R) = \int r_{ij}p(r_{ij}|R)dr_{ij}
$$

$$
E(r_{ij}|R) = \int r_{ij}p(r_{ij}|W,U) p(W,U|R)dWdUdr_{ij}
$$

- $p(r_{ij}|W,U)$ : our original gaussian
- $p(W,U|R)$ : posterior

$$
\int r_{ij}p(r_{ij}|W,U) dr_{ij} = E(r_{ij}|W,U) = w_{i}^{T}u_{j}\\
r_{ij} \sim N(w_{i}^{T}u_{j},\sigma^{2})
$$

$$
E(r_{ij}|R)= \int w_{i}^{T}u_{j} p(W,U|R)dWdU \\
E(r_{ij}|R)= E(w_{i}^{T}u_{j}|R)
$$

- we can approximate this expected value
- A mean can be approximated by the sample mean, if we sample from the correct distribution
- MCMC(Markov Chain MonteCarlo) - Gibbs Sampling

$$
E(r_{ij}|R) \approx \frac{1}{T}\sum_{t=1}^{T}w_{i}^{(t)T}u_{j}^{(t)}
$$





[bpmf paper](https://www.cs.toronto.edu/~amnih/papers/bpmf.pdf)

# 10) Matrix Factorization In Keras

- Algorithm for training neural nets: gradient descent


## Gradient Descent for Matirx Factorization
- Both gradient descent and alternating least squares are valid training algorithms for matrix factorization


## Embedding
- N = (vocabulary size) K =(feature vector size)
- N x K matrix

```python
emb_u = Embedding(N,K)
emb_m = Embedding(M,K)

#inputs
u = Input() #user
m = Input() #movie

eu = emb_u(u) # returns a K-size vector
em = emb_m(m) 

# dot them to get prediction
r = dot(eu,em)
```

## Bias terms
- not straightforward
- keras is not a low level library
- Embedding(N,1) gives me a scalar for a user bias

```python
bias_u = Embedding(N,1)
bias_m = Embedding(N,1)

bu = bias_u(u)
bm = bias_u(m)

r = add(r,bu,bm) # r = dot(eu,em)

```

## global average
- we want the model $r_{ij} = dot(w_{i},u_{j}) + b_{i}+c_{j}+mu$
- simple solution
- make the target = actual ratings - mu


# 11) Matrix Factorization In Keras In Code

```python
# load in the data
df = pd.read_csv('../large_files/movielens-20m-dataset/edited_rating.csv')

N = df.userId.max() + 1 # number of users
M = df.movie_idx.max() + 1 # number of movies

```

```python
# initialize variables
K = 10 # latent dimensionality
mu = df_train.rating.mean()
epochs = 15
reg = 0. # regularization penalty

```

```python

# keras model
u = Input(shape=(1,))
m = Input(shape=(1,))
u_embedding = Embedding(N, K, embeddings_regularizer=l2(reg))(u) # (N, 1, K)
m_embedding = Embedding(M, K, embeddings_regularizer=l2(reg))(m) # (N, 1, K)
```

```python
'''
u_embedding,m_embedding
(<tf.Tensor 'embedding_1/GatherV2:0' shape=(?, 1, 5) dtype=float32>,
 <tf.Tensor 'embedding_2/GatherV2:0' shape=(?, 1, 5) dtype=float32>)
''' 
```

```python
u_bias = Embedding(N, 1, embeddings_regularizer=l2(reg))(u) # (N, 1, 1)
m_bias = Embedding(M, 1, embeddings_regularizer=l2(reg))(m) # (N, 1, 1)
x = Dot(axes=2)([u_embedding, m_embedding]) # (N, 1, 1)

```

```python
x = Add()([x, u_bias, m_bias])
x = Flatten()(x) # (N, 1)
```

```python
model = Model(inputs=[u, m], outputs=x)
model.compile(
  loss='mse',
  # optimizer='adam',
  # optimizer=Adam(lr=0.01),
  optimizer=SGD(lr=0.08, momentum=0.9),
  metrics=['mse'],
)

r = model.fit(
  x=[df_train.userId.values, df_train.movie_idx.values],
  y=df_train.rating.values - mu,
  epochs=epochs,
  batch_size=128,
  validation_data=(
    [df_test.userId.values, df_test.movie_idx.values],
    df_test.rating.values - mu
  )
)
```

# 12) Deep Learning

- Input is a feature vector

- we already have a feature vector that represents the user, and a feature vector that represents the item
- User vector: W[i]
- Movie vector: U[j]  

\begin{array}
 UUser & \xrightarrow{} & Embedding & \xrightarrow{} & Concat & \xrightarrow{} & Neural Network\\
 Movie & \xrightarrow{} & Embedding & \nearrow & \\
\end{array}


- since it's regression, final layer is Dense(1) with no activation function

## Hyperparameter 
- number layers
- number hidden units
- hidden activation
- optimizer
- ...

```python
# keras model
u = Input(shape=(1,))
m = Input(shape=(1,))
u_embedding = Embedding(N, K)(u) # (N, 1, K)
m_embedding = Embedding(M, K)(m) # (N, 1, K)
u_embedding = Flatten()(u_embedding) # (N, K)
m_embedding = Flatten()(m_embedding) # (N, K)
x = Concatenate()([u_embedding, m_embedding]) # (N, 2K)

# the neural network
x = Dense(400)(x)
# x = BatchNormalization()(x)
x = Activation('relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(100)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
x = Dense(1)(x)

model = Model(inputs=[u, m], outputs=x)
model.compile(
  loss='mse',
  # optimizer='adam',
  # optimizer=Adam(lr=0.01),
  optimizer=SGD(lr=0.08, momentum=0.9),
  metrics=['mse'],
)

```

# 13) Residual Learning

## Computer vision Inspiration
- An architecture inspired by residual networks and inception
- Idea: different branches will identify different patterns


## Why apply to recommenders?
- We already know MF works well(but it's linear)
- we know that a deep neural network can find nonlinear patterns

$$
\hat{r} = MF(x) + ANN(x)\\
ANN(x) = \hat{x} - MF(x) = error\; made\; by\; MF\;   model(residual) 
$$

- Just need one more Add() layer to join them


```python
# initialize variables
K = 10 # latent dimensionality
mu = df_train.rating.mean()
epochs = 15
reg = 0. # regularization penalty


# keras model
u = Input(shape=(1,))
m = Input(shape=(1,))
u_embedding = Embedding(N, K)(u) # (N, 1, K)
m_embedding = Embedding(M, K)(m) # (N, 1, K)


##### main branch
u_bias = Embedding(N, 1)(u) # (N, 1, 1)
m_bias = Embedding(M, 1)(m) # (N, 1, 1)
x = Dot(axes=2)([u_embedding, m_embedding]) # (N, 1, 1)
x = Add()([x, u_bias, m_bias])
x = Flatten()(x) # (N, 1)


##### side branch
u_embedding = Flatten()(u_embedding) # (N, K)
m_embedding = Flatten()(m_embedding) # (N, K)
y = Concatenate()([u_embedding, m_embedding]) # (N, 2K)
y = Dense(400)(y)
y = Activation('elu')(y)
# y = Dropout(0.5)(y)
y = Dense(1)(y)


##### merge
x = Add()([x, y])

model = Model(inputs=[u, m], outputs=x)
model.compile(
  loss='mse',
  # optimizer='adam',
  # optimizer=Adam(lr=0.01),
  optimizer=SGD(lr=0.08, momentum=0.9),
  metrics=['mse'],
)

r = model.fit(
  x=[df_train.userId.values, df_train.movie_idx.values],
  y=df_train.rating.values - mu,
  epochs=epochs,
  batch_size=128,
  validation_data=(
    [df_test.userId.values, df_test.movie_idx.values],
    df_test.rating.values - mu
  )
)

```

# 14) AutoEncoder

## AutoRec
- Application of autoencoder to recommendations
- AutoRec - Autoencoder Meet Collaborative Filtering

- A neural network that just reproduces its own input
- Error = $(Output\; -\; input)^2$

```python
ann = Model()
ann.fit(X,X)
```

## Denoising Autoencoders

![](https://miro.medium.com/max/3486/1*G0V4dz4RKTKGpebeoSWB0A.png)

## Application to Recommenders
- That's exactly what we're trying to do
- The missing ratings are what we're trying to predict

## Image Data for a Denoising Autoencoder

- Still N x D (# samples x # features)

[]()|Pixel 1|...|Pixel D|
--|--|--|--|
Image 1|25|...|1|
Image 2|0|...|0|
Image 3|3|...|0|
Image 4|10|...|1|

## Ratings Matrix
- Each individual cell is a sample
- e.g(User1, Item1, 5), (User1, Item3,1),...
- We should have 20 million samples(because there are 20 million ratings)

[]()|Item 1|Item 2|Item 3|
--|--|--|--|
User 1|5|[]()|1|
User 2|[]()|3|[]()|
User 3|3|2|[]()|
User 4|4|2|1|

## AutoRec
- Simply treat the user-movies matrix as if it were an samples-features data matrix(with tons of missing values)
- User your imagination
    - Each user is a sample
    - Each feature is a movie rating
- You can flip it on its side and do it the other way
    - But authors found this way works better
    

## New Additions
- add you own noise to inputs
- goal: you want to predict missing ratings
- Not simply reconstruct input(predict ratings that are already given)
- simple in keras: Dropout()


## N x M matrix
- if the ratings matrix is now our training data, how can we pass it into neural network if we can't store an NxM matrix in memory?
- Special format in Scipy: sparse matrix
- lil_matrix, csr_matrix, coo_matrix
- preprocess2sparse.py
- Keras doesn't recongnize sparse matrix
- solution: custom generator
    - densify only a batch at a time
    

## Calculating cost
- An array can't contain missing values, just zeros(luckily, all actual ratings are 0.5- 5.0, so it's easy to differentiate)
- Must make sure autoencoder doesn't literally try to reproduce input x(otherwise, it will try to reproduce the zeros too)
- solution: mask the cost

$$
J = \frac{1}{|\Omega|}\sum_{i=1}^{N}\sum_{j=1}^{M}m_{ij}(r_{ij}-\hat{r_{ij}})^2 \\
m_{ij} = 1 \; if(i,j)\in\Omega\; else\; 0
$$

## Test MSE

```python
pred = model.predict(X_train) # output is N x M
error = get_loss(X_test,pred)
```


$$
J_{test} = \frac{1}{|\Omega_{test}|}\sum_{i=1}^{N}\sum_{j=1}^{M}m_{ij}^{(test)}(r_{ij}-\hat{r_{ij}})^2 \\
m_{ij}^{(test)} = 1 \; if(i,j)\in\Omega_{test}\; else\; 0\\
\hat{r}_{ij} = reconstruction\; from\; training\; data
$$

# 15) AutoEncoder in Code



In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from scipy.sparse import lil_matrix, csr_matrix, save_npz, load_npz


# load in the data
df = pd.read_csv('./data/movielens-20m-dataset/rating.csv')
# df = pd.read_csv('../large_files/movielens-20m-dataset/small_rating.csv')

N = df.userId.max() + 1 # number of users
M = df.movieId.max() + 1 # number of movies

# split into train and test
df = shuffle(df)
cutoff = int(0.8*len(df))
df_train = df.iloc[:cutoff]
df_test = df.iloc[cutoff:]

# create sparse matrix (N,M)
A = lil_matrix((N, M))


In [ ]:
count = 0
def update_train(row):
  global count
  count += 1
  if count % 100000 == 0:
    print("processed: %.3f" % (float(count)/cutoff))

  i = int(row.userId)
  j = int(row.movieId)
  A[i,j] = row.rating
df_train.apply(update_train, axis=1)


In [11]:
# mask, to tell us which entries exist and which do not
A = A.tocsr() # Convert this matrix to Compressed Sparse Row format.
mask = (A > 0)
save_npz("Atrain.npz", A)

In [19]:
A.get_shape()

(138494, 131263)

### Autoencoder AutoRec
```python
# config
batch_size = 128
epochs = 20
reg = 0.0001
# reg = 0

A = load_npz("Atrain.npz")
A_test = load_npz("Atest.npz")
mask = (A > 0) * 1.0
mask_test = (A_test > 0) * 1.0
```

```python
# make copies since we will shuffle
A_copy = A.copy()
mask_copy = mask.copy()
A_test_copy = A_test.copy()
mask_test_copy = mask_test.copy()
```

```python
# (138494, 131263)
N, M = A.shape
# center the data, global average
mu = A.sum() / mask.sum()
```

```python
# build the model - just a 1 hidden layer autoencoder
i = Input(shape=(M,))
# bigger hidden layer size seems to help!
x = Dropout(0.7)(i) # like Denoising autoencoder
x = Dense(700, activation='tanh', kernel_regularizer=l2(reg))(x)
# x = Dropout(0.5)(x)
x = Dense(M, kernel_regularizer=l2(reg))(x)
```


$$
J = \frac{1}{|\Omega|}\sum_{i=1}^{N}\sum_{j=1}^{M}m_{ij}(r_{ij}-\hat{r_{ij}})^2 \\
m_{ij} = 1 \; if(i,j)\in\Omega\; else\; 0
$$

```python

def custom_loss(y_true, y_pred):
  mask = K.cast(K.not_equal(y_true, 0), dtype='float32')
  diff = y_pred - y_true
  sqdiff = diff * diff * mask
  sse = K.sum(K.sum(sqdiff))
  n = K.sum(K.sum(mask))
  return sse / n
```

```python

def generator(A, M):
  while True:
    A, M = shuffle(A, M)
    for i in range(A.shape[0] // batch_size + 1):
      upper = min((i+1)*batch_size, A.shape[0])
      a = A[i*batch_size:upper].toarray()
      m = M[i*batch_size:upper].toarray()
      a = a - mu * m # must keep zeros at zero!
      # m2 = (np.random.random(a.shape) > 0.5)
      # noisy = a * m2
      noisy = a # no noise
      yield noisy, a
        
```




$$
J_{test} = \frac{1}{|\Omega_{test}|}\sum_{i=1}^{N}\sum_{j=1}^{M}m_{ij}^{(test)}(r_{ij}-\hat{r_{ij}})^2 \\
m_{ij}^{(test)} = 1 \; if(i,j)\in\Omega_{test}\; else\; 0\\
\hat{r}_{ij} = reconstruction\; from\; training\; data
$$
```python
def test_generator(A, M, A_test, M_test):
  # assumes A and A_test are in corresponding order
  # both of size N x M
  while True:
    for i in range(A.shape[0] // batch_size + 1):
      upper = min((i+1)*batch_size, A.shape[0])
      a = A[i*batch_size:upper].toarray()
      m = M[i*batch_size:upper].toarray()
      at = A_test[i*batch_size:upper].toarray()
      mt = M_test[i*batch_size:upper].toarray()
      a = a - mu * m
      at = at - mu * mt
      yield a, at
        
```

```python

model = Model(i, x)
model.compile(
  loss=custom_loss,
  optimizer=SGD(lr=0.08, momentum=0.9),
  # optimizer='adam',
  metrics=[custom_loss],
)


r = model.fit_generator(
  generator(A, mask),
  validation_data=test_generator(A_copy, mask_copy, A_test_copy, mask_test_copy),
  epochs=epochs,
  steps_per_epoch=A.shape[0] // batch_size + 1,
  validation_steps=A_test.shape[0] // batch_size + 1,
)
```

![](https://raw.githubusercontent.com/databricks/spark-training/master/website/img/matrix_factorization.png)